Determine if the patient will die immediately (including died within 24h out of the hospital), based on the medical tests, and other personal info.


Focus on patients that are:

age > 16

icu_times_total = 1 or (icu_times_total > 1 and died_imm) 

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

sqluser = 'postgres'
dbname = 'mimic_demo'
schema_name = 'mimiciii'
password = '012500'
con = psycopg2.connect(dbname=dbname, user=sqluser, password=password)
query_schema = 'set search_path to ' + schema_name + ';'

get patient personal information:age,death time, if died...

In [13]:
query_personal_info = """
WITH patient_icu_age AS (
  SELECT
    i.subject_id,
    i.hadm_id,
    i.icustay_id,
    p.gender,
    ROUND(EXTRACT(DAY FROM i.intime - p.dob) / 365.25, 2) AS age_inICU_year,  -- Using 365.25 to account for leap years
    (EXTRACT(DAY FROM i.intime - p.dob) * 1440) + 
    (EXTRACT(HOUR FROM i.intime - p.dob) * 60) + 
    EXTRACT(MINUTE FROM i.intime - p.dob) AS age_inICU_min,
    i.intime,
    i.los as icu_duration_hour,
    p.expire_flag
  FROM
    mimiciii.icustays i
    LEFT JOIN mimiciii.patients p ON i.subject_id = p.subject_id
),

patient_icu_personal_info_withoutICUTimesTotal AS (
  SELECT
    pa.subject_id,
    pa.hadm_id,
    pa.icustay_id,
    pa.gender,
    pa.age_inICU_year,
    pa.age_inICU_min,
    a.INSURANCE,
    a.RELIGION,
    a.MARITAL_STATUS,
    a.ETHNICITY,
    a.DIAGNOSIS,
    a.DISCHTIME,
    a.DEATHTIME,
    pa.intime,
    RANK() OVER (PARTITION BY pa.subject_id ORDER BY pa.intime) AS icu_times,
    pa.icu_duration_hour,
    a.HOSPITAL_EXPIRE_FLAG,
    pa.expire_flag,
    CASE 
      WHEN a.deathtime is null or EXTRACT(DAY FROM a.deathtime - a.dischtime) > 1 THEN 0
      ELSE 1
    END AS died_immediately
  FROM
    patient_icu_age pa
    LEFT JOIN mimiciii.admissions a ON pa.hadm_id = a.hadm_id 
),

patient_icu_personal_info_all as (
  SELECT *, max(icu_times) over (partition by subject_id) as icu_times_total
  FROM patient_icu_personal_info_withoutICUTimesTotal
)

select * from patient_icu_personal_info_all
where icu_times_total > 1

;

"""
patient_icuid_age = pd.read_sql_query(query_personal_info, con)
patient_icuid_age.head(10)

,subject_id,hadm_id,icustay_id,gender,age_inicu_year,age_inicu_min,insurance,religion,marital_status,ethnicity,diagnosis,dischtime,deathtime,intime,icu_times,icu_duration_hour,hospital_expire_flag,expire_flag,died_immediately,icu_times_total
0,10059,142582,215460,M,69.59,36601781.0,Medicare,CATHOLIC,MARRIED,WHITE,VARICEAL BLEED,2150-08-13 11:33:00,NaT,2150-08-07 21:41:28,1,1.7806,0,1,0,2
1,10059,122098,248755,M,69.63,36623133.0,Medicare,CATHOLIC,MARRIED,WHITE,LOWER GI BLEED,2150-08-29 18:20:00,2150-08-29 18:20:00,2150-08-22 17:33:50,2,7.1083,1,1,1,2
2,10088,169938,256345,M,77.49,40756323.0,Medicare,UNOBTAINABLE,UNKNOWN (DEFAULT),WHITE,SEPSIS;PNEUMONIA;TELEMETRY,2107-01-11 15:45:00,NaT,2107-01-05 00:03:45,1,3.6885,0,1,0,3
3,10088,168233,249695,M,77.56,40791122.0,Medicare,UNOBTAINABLE,UNKNOWN (DEFAULT),WHITE,CONGESTIVE HEART FAILURE,2107-02-10 12:00:00,NaT,2107-01-29 04:02:15,2,1.6226,0,1,0,3
4,10088,149044,277403,M,77.84,40940280.0,Private,UNOBTAINABLE,UNKNOWN (DEFAULT),WHITE,UROSEPSIS,2107-05-18 13:30:00,NaT,2107-05-12 18:00:52,3,2.0091,0,1,0,3
5,10094,168074,243600,M,299.99,157786254.0,Medicare,UNOBTAINABLE,None,BLACK/AFRICAN AMERICAN,HYPOTENSION;TELEMETRY,2180-03-10 17:35:00,NaT,2180-02-29 18:54:39,1,4.1014,0,1,0,2
6,10094,122928,273347,M,300.04,157808075.0,Medicare,UNOBTAINABLE,None,BLACK/AFRICAN AMERICAN,SEPSIS;TELEMETRY,2180-03-20 18:00:00,2180-03-20 18:00:00,2180-03-15 22:35:30,2,3.9712,1,1,1,2
7,10117,187023,214861,F,66.08,34757056.0,Private,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,FEVER,2138-06-11 10:16:00,NaT,2138-06-05 20:16:00,1,1.7861,0,1,0,2
8,10117,105150,282566,F,66.52,34986438.0,Private,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,FEVER,2138-11-18 23:13:00,2138-11-18 23:13:00,2138-11-12 03:18:24,2,2.6745,1,1,1,2
9,10119,157466,247686,M,81.40,42815345.0,Medicare,NOT SPECIFIED,MARRIED,WHITE,ACUTE CHOLECYSTITIS,2117-08-19 16:15:00,NaT,2117-08-05 21:05:40,1,7.9020,0,1,0,2


Get all the medical tests the patient did.

In [14]:
# get all medical tests (test name, test code, test value, test time) all icustay_id had, 
# where the test is completly conducted

query_medtest_rec = '''
    select 
    c.SUBJECT_ID,
    c.HADM_ID,
    c.ICUSTAY_ID,
    c.ITEMID,
    di.label,
    c.CHARTTIME,
    c.VALUE
    
    from mimiciii.chartevents c left join mimiciii.d_items di on c.ITEMID = di.ITEMID
    where c.stopped = 'NotStopd';
    
'''
patient_medtest_rec = pd.read_sql_query(query_medtest_rec, con)
patient_medtest_rec.head(10)


,subject_id,hadm_id,icustay_id,itemid,label,charttime,value
0,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 01:00:00,1.4
1,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 02:00:00,1.4
2,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 03:00:00,1.4
3,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 04:00:00,1.4
4,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 05:00:00,1.4
5,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 06:00:00,1.4
6,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 07:00:00,1.4
7,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 08:00:00,1.4
8,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 10:00:00,1.4
9,10019,177759,228977.0,24,ART Lumen Volume,2163-05-15 11:00:00,1.4


Get patients' output records which might be useful.

In [15]:
query_output_rec = '''
 select 
    o.SUBJECT_ID,
    o.HADM_ID,
    o.ICUSTAY_ID,
    o.ITEMID,
    di.label,
    o.CHARTTIME,
    cast (o.VALUE as character varying) as value
    
    from mimiciii.outputevents o left join mimiciii.d_items di on o.ITEMID = di.ITEMID
'''

patient_output_rec = pd.read_sql_query(query_output_rec, con)
patient_output_rec.head(10)

,subject_id,hadm_id,icustay_id,itemid,label,charttime,value
0,10114,167957,234989.0,40055,Urine Out Foley,2171-10-30 20:00:00,39
1,10114,167957,234989.0,40055,Urine Out Foley,2171-10-30 21:00:00,35
2,10114,167957,234989.0,40055,Urine Out Foley,2171-10-30 23:00:00,100
3,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 00:00:00,45
4,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 02:00:00,80
5,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 03:00:00,80
6,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 05:00:00,140
7,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 06:00:00,220
8,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 08:00:00,220
9,10114,167957,234989.0,40055,Urine Out Foley,2171-10-31 10:00:00,200


Union med test records and output records.

In [16]:
query_allMedTest_rec = '''
    with medTestRec as (
        select 
        c.SUBJECT_ID,
        c.HADM_ID,
        c.ICUSTAY_ID,
        c.ITEMID,
        di.label,
        c.CHARTTIME,
        c.VALUE

        from mimiciii.chartevents c left join mimiciii.d_items di on c.ITEMID = di.ITEMID
        where c.stopped = 'NotStopd'
    ),
    
    outputRec as (
        select 
        o.SUBJECT_ID,
        o.HADM_ID,
        o.ICUSTAY_ID,
        o.ITEMID,
        di.label,
        o.CHARTTIME,
        cast (o.VALUE as character varying) as value
    
        from mimiciii.outputevents o left join mimiciii.d_items di on o.ITEMID = di.ITEMID
    ),
    
    allTestRec as (
        select subject_id, hadm_id, icustay_id, itemid, label, charttime, value 
        from medTestRec
        union
        select subject_id, hadm_id, icustay_id, itemid, label, charttime, value 
        from outputRec)
    
	    select *from allTestRec
        
'''
patient_allTestRec = pd.read_sql_query(query_allMedTest_rec, con)
patient_allTestRec.head(20)


,subject_id,hadm_id,icustay_id,itemid,label,charttime,value
0,10006,142345,206504.0,27,Abdominal Assessment,2164-10-23 23:00:00,Soft
1,10006,142345,206504.0,27,Abdominal Assessment,2164-10-24 04:00:00,Soft
2,10006,142345,206504.0,27,Abdominal Assessment,2164-10-24 08:00:00,Soft
3,10006,142345,206504.0,27,Abdominal Assessment,2164-10-24 12:00:00,Soft
4,10006,142345,206504.0,27,Abdominal Assessment,2164-10-24 16:00:00,Soft
5,10006,142345,206504.0,27,Abdominal Assessment,2164-10-24 20:00:00,Soft
6,10006,142345,206504.0,27,Abdominal Assessment,2164-10-25 00:00:00,Soft
7,10006,142345,206504.0,27,Abdominal Assessment,2164-10-25 06:00:00,Soft
8,10006,142345,206504.0,31,Activity,2164-10-23 23:00:00,Bedrest
9,10006,142345,206504.0,31,Activity,2164-10-24 02:00:00,Bedrest


Union all, the raw dataset we need. Further filter actions will be done.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

sqluser = 'postgres'
dbname = 'mimic_demo'
schema_name = 'mimiciii'
password = '012500'
con = psycopg2.connect(dbname=dbname, user=sqluser, password=password)
query_schema = 'set search_path to ' + schema_name + ';'

In [21]:
patient_complete_query = '''
WITH patient_icu_age AS (
  SELECT
    i.subject_id,
    i.hadm_id,
    i.icustay_id,
    p.gender,
    ROUND(EXTRACT(DAY FROM i.intime - p.dob) / 365.25, 2) AS age_inICU_year,  -- Using 365.25 to account for leap years
    (EXTRACT(DAY FROM i.intime - p.dob) * 1440) + 
    (EXTRACT(HOUR FROM i.intime - p.dob) * 60) + 
    EXTRACT(MINUTE FROM i.intime - p.dob) AS age_inICU_min,
    i.intime,
    i.los as icu_duration_hour,
    p.expire_flag
  FROM
    mimiciii.icustays i
    LEFT JOIN mimiciii.patients p ON i.subject_id = p.subject_id
),

patient_icu_personal_info_withoutICUTimesTotal AS (
  SELECT
    pa.subject_id,
    pa.hadm_id,
    pa.icustay_id,
    pa.gender,
    pa.age_inICU_year,
    pa.age_inICU_min,
    a.INSURANCE,
    a.RELIGION,
    a.MARITAL_STATUS,
    a.ETHNICITY,
    a.DIAGNOSIS,
    a.DISCHTIME,
    a.DEATHTIME,
    pa.intime,
    RANK() OVER (PARTITION BY pa.subject_id ORDER BY pa.intime) AS icu_times,
    pa.icu_duration_hour,
    a.HOSPITAL_EXPIRE_FLAG,
    pa.expire_flag,
    CASE 
      WHEN a.deathtime is null or EXTRACT(DAY FROM a.deathtime - a.dischtime) > 1 THEN 0
      ELSE 1
    END AS died_immediately
  FROM
    patient_icu_age pa
    LEFT JOIN mimiciii.admissions a ON pa.hadm_id = a.hadm_id 
),

patient_icu_personal_info_all as (
  SELECT *, max(icu_times) over (partition by subject_id) as icu_times_total
  FROM patient_icu_personal_info_withoutICUTimesTotal
),

medTestRec as (
        select 
        c.SUBJECT_ID,
        c.HADM_ID,
        c.ICUSTAY_ID,
        c.ITEMID,
        di.label,
        c.CHARTTIME,
        c.VALUE

        from mimiciii.chartevents c left join mimiciii.d_items di on c.ITEMID = di.ITEMID
        where c.stopped = 'NotStopd'
    ),
    
    outputRec as (
        select 
        o.SUBJECT_ID,
        o.HADM_ID,
        o.ICUSTAY_ID,
        o.ITEMID,
        di.label,
        o.CHARTTIME,
        cast (o.VALUE as character varying) as value
    
        from mimiciii.outputevents o left join mimiciii.d_items di on o.ITEMID = di.ITEMID
    ),
    
    allTestRec as (
        select subject_id, hadm_id, icustay_id, itemid, label, charttime, value 
        from medTestRec
        union
        select subject_id, hadm_id, icustay_id, itemid, label, charttime, value 
        from outputRec),
    
    patient_complete as (
      SELECT
      pi.subject_id,
      pi.hadm_id,
      pi.icustay_id,
      pi.age_inICU_year,
      pi.age_inICU_min,
      pi.INSURANCE,
      pi.gender,
      pi.RELIGION,
      pi.MARITAL_STATUS,
      pi.ETHNICITY,
      pi.DIAGNOSIS,
      pi.DISCHTIME,
      pi.DEATHTIME,
      pi.icu_duration_hour,
      at.itemid,
      at.label as test_label,
      at.charttime as test_time,
      pi.intime,
      pi.icu_times,
      pi.icu_times_total,
      
      extract(day from at.charttime - pi.intime) * 24 +
      extract(hour from at.charttime - pi.intime) as test_time_from_inICU_hour,
      
      extract(day from at.charttime - pi.intime) * 24 * 60 +
      extract(hour from at.charttime - pi.intime) * 60 +
      extract(min from at.charttime - pi.intime) as test_time_from_inICU_min, 
      at.value as test_value,
      
      pi.HOSPITAL_EXPIRE_FLAG,
      pi.expire_flag,
      pi.died_immediately
            
      from allTestRec at 
      left join
      patient_icu_personal_info_all pi
      on at.icustay_id = pi.icustay_id
    )
    
    select * from patient_complete
    where diagnosis ~* 'sepsis'
    
'''

patient_complete = pd.read_sql_query(patient_complete_query, con)
#patient_sepsis_complete.to_csv(r'F:\STUDY\python_code\Pract_DA\prepocess\patient_sepsis_complete.csv')
patient_complete

,subject_id,hadm_id,icustay_id,age_inicu_year,age_inicu_min,insurance,gender,religion,marital_status,ethnicity,...,test_time,intime,icu_times,icu_times_total,test_time_from_inicu_hour,test_time_from_inicu_min,test_value,hospital_expire_flag,expire_flag,died_immediately
0,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,2164-10-23 23:00:00,2164-10-23 21:10:15,1,1,1.0,109.0,Soft,0,1,0
1,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,2164-10-24 04:00:00,2164-10-23 21:10:15,1,1,6.0,409.0,Soft,0,1,0
2,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,2164-10-24 08:00:00,2164-10-23 21:10:15,1,1,10.0,649.0,Soft,0,1,0
3,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,2164-10-24 12:00:00,2164-10-23 21:10:15,1,1,14.0,889.0,Soft,0,1,0
4,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,2164-10-24 16:00:00,2164-10-23 21:10:15,1,1,18.0,1129.0,Soft,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20060,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,2201-05-13 14:00:00,2201-05-12 10:50:34,7,15,27.0,1629.0,25,0,1,0
20061,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,2201-05-13 16:00:00,2201-05-12 10:50:34,7,15,29.0,1749.0,240,0,1,0
20062,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,2201-05-13 17:00:00,2201-05-12 10:50:34,7,15,30.0,1809.0,60,0,1,0
20063,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,2201-05-13 18:00:00,2201-05-12 10:50:34,7,15,31.0,1869.0,120,0,1,0


consider sepsis only

In [22]:
patient_sepsis_query = '''
WITH patient_icu_age AS (
  SELECT
    i.subject_id,
    i.hadm_id,
    i.icustay_id,
    p.gender,
    ROUND(EXTRACT(DAY FROM i.intime - p.dob) / 365.25, 2) AS age_inICU_year,  -- Using 365.25 to account for leap years
    (EXTRACT(DAY FROM i.intime - p.dob) * 1440) + 
    (EXTRACT(HOUR FROM i.intime - p.dob) * 60) + 
    EXTRACT(MINUTE FROM i.intime - p.dob) AS age_inICU_min,
    i.intime,
    i.los as icu_duration_hour,
    p.expire_flag
  FROM
    mimiciii.icustays i
    LEFT JOIN mimiciii.patients p ON i.subject_id = p.subject_id
),

patient_icu_personal_info_withoutICUTimesTotal_all AS (
  SELECT
    pa.subject_id,
    pa.hadm_id,
    pa.icustay_id,
    pa.gender,
    pa.age_inICU_year,
    pa.age_inICU_min,
    a.INSURANCE,
    a.RELIGION,
    a.MARITAL_STATUS,
    a.ETHNICITY,
    a.DIAGNOSIS,
    a.DISCHTIME,
    a.DEATHTIME,
    pa.intime,
    RANK() OVER (PARTITION BY pa.subject_id ORDER BY pa.intime) AS icu_times,
    pa.icu_duration_hour,
    a.HOSPITAL_EXPIRE_FLAG,
    pa.expire_flag,
    CASE 
      WHEN a.deathtime is null or EXTRACT(DAY FROM a.deathtime - a.dischtime) > 1 THEN 0
      ELSE 1
    END AS died_immediately
  FROM
    patient_icu_age pa
    LEFT JOIN mimiciii.admissions a ON pa.hadm_id = a.hadm_id 
),

patient_icu_personal_info_withoutICUTimesTotal_sepsis AS (
  SELECT
    pa.subject_id,
    pa.hadm_id,
    pa.icustay_id,
    RANK() OVER (PARTITION BY pa.subject_id ORDER BY pa.intime) AS icu_times_sepsis
  FROM
    patient_icu_age pa
    LEFT JOIN mimiciii.admissions a ON pa.hadm_id = a.hadm_id 
    where a.diagnosis ~*'sepsis'
),

patient_icu_personal_info_withoutICUTimesTotal as (
    select
    paa.subject_id,
    paa.hadm_id,
    paa.icustay_id,
    paa.gender,
    paa.age_inICU_year,
    paa.age_inICU_min,
    paa.INSURANCE,
    paa.RELIGION,
    paa.MARITAL_STATUS,
    paa.ETHNICITY,
    paa.DIAGNOSIS,
    paa.DISCHTIME,
    paa.DEATHTIME,
    paa.intime,
    paa.icu_times,
    pas.icu_times_sepsis,
    paa.icu_duration_hour,
    paa.HOSPITAL_EXPIRE_FLAG,
    paa.expire_flag,
    paa.died_immediately
    
    from patient_icu_personal_info_withoutICUTimesTotal_all paa
    left join  patient_icu_personal_info_withoutICUTimesTotal_sepsis pas
    on paa.icustay_id = pas.icustay_id
    
  
),

patient_icu_personal_info_all as (
  SELECT *, 
  max(icu_times) over (partition by subject_id) as icu_times_total,
  max(icu_times_sepsis) over (partition by subject_id) as icu_times_total_sepsis
  FROM patient_icu_personal_info_withoutICUTimesTotal
),

medTestRec as (
        select 
        c.SUBJECT_ID,
        c.HADM_ID,
        c.ICUSTAY_ID,
        c.ITEMID,
        di.label,
        c.CHARTTIME,
        c.VALUE

        from mimiciii.chartevents c left join mimiciii.d_items di on c.ITEMID = di.ITEMID
        where c.stopped = 'NotStopd'
    ),
    
    outputRec as (
        select 
        o.SUBJECT_ID,
        o.HADM_ID,
        o.ICUSTAY_ID,
        o.ITEMID,
        di.label,
        o.CHARTTIME,
        cast (o.VALUE as character varying) as value
    
        from mimiciii.outputevents o left join mimiciii.d_items di on o.ITEMID = di.ITEMID
    ),
    
    allTestRec as (
        select subject_id, hadm_id, icustay_id, itemid, label, charttime, value 
        from medTestRec
        union
        select subject_id, hadm_id, icustay_id, itemid, label, charttime, value 
        from outputRec),
    
    patient_complete_sepsis as (
      SELECT
      pi.subject_id,
      pi.hadm_id,
      pi.icustay_id,
      pi.age_inICU_year,
      pi.age_inICU_min,
      pi.INSURANCE,
      pi.gender,
      pi.RELIGION,
      pi.MARITAL_STATUS,
      pi.ETHNICITY,
      pi.DIAGNOSIS,
      pi.DISCHTIME,
      pi.DEATHTIME,
      pi.icu_duration_hour,
      at.itemid,
      at.label as test_label,
      at.charttime as test_time,
      pi.intime,
      pi.icu_times,
      pi.icu_times_total,
      pi.icu_times_sepsis,
      pi.icu_times_total_sepsis,
      
      extract(day from at.charttime - pi.intime) * 24 +
      extract(hour from at.charttime - pi.intime) as test_time_from_inICU_hour,
      
      extract(day from at.charttime - pi.intime) * 24 * 60 +
      extract(hour from at.charttime - pi.intime) * 60 +
      extract(min from at.charttime - pi.intime) as test_time_from_inICU_min, 
      at.value as test_value,
      
      pi.HOSPITAL_EXPIRE_FLAG,
      pi.expire_flag,
      pi.died_immediately
            
      from allTestRec at
      left join
      patient_icu_personal_info_all pi
      on at.icustay_id = pi.icustay_id
    )
    
    

    select * from patient_complete_sepsis
    where diagnosis ~* 'sepsis'
    
    
'''

patient_sepsis = pd.read_sql_query(patient_sepsis_query, con)
patient_sepsis

#patient_sepsis['comorbility'] 


,subject_id,hadm_id,icustay_id,age_inicu_year,age_inicu_min,insurance,gender,religion,marital_status,ethnicity,...,icu_times,icu_times_total,icu_times_sepsis,icu_times_total_sepsis,test_time_from_inicu_hour,test_time_from_inicu_min,test_value,hospital_expire_flag,expire_flag,died_immediately
0,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,1,1,1,1,1.0,109.0,Soft,0,1,0
1,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,1,1,1,1,6.0,409.0,Soft,0,1,0
2,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,1,1,1,1,10.0,649.0,Soft,0,1,0
3,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,1,1,1,1,14.0,889.0,Soft,0,1,0
4,10006,142345,206504,70.63,37151830.0,Medicare,F,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,...,1,1,1,1,18.0,1129.0,Soft,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20060,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,7,15,2,6,27.0,1629.0,25,0,1,0
20061,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,7,15,2,6,29.0,1749.0,240,0,1,0
20062,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,7,15,2,6,30.0,1809.0,60,0,1,0
20063,41976,180546,285353,64.79,34075370.0,Medicare,M,CATHOLIC,MARRIED,HISPANIC/LATINO - PUERTO RICAN,...,7,15,2,6,31.0,1869.0,120,0,1,0


A new table in DATASET> mimic_demo SCHEMA> mimiciii_generate TABLE>patient_complete

NOW move on to raw_dataStats_demo